In [ ]:
# === RealSense + YOLO + PaddleOCR + GPT 교정 (형광 색상 통합 최종 시연버전) ===
import os, gc, re, cv2, torch, numpy as np, matplotlib.pyplot as plt
import pyrealsense2 as rs
from ultralytics import YOLO
from paddleocr import PaddleOCR
from difflib import SequenceMatcher
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from PIL import ImageFont, ImageDraw, Image
from openai import OpenAI

# ===========================
#   기본 설정
# ===========================
BAG_FILE = r"/workspace/data/gp2.bag"
YOLO_WEIGHTS = r"/workspace/mk/yolo11m-seg.pt"
FONT_PATH = "/workspace/nk/1_캡스톤디자인/NanumGothicCoding-Bold.ttf"

FRAME_START, FRAME_END = 0, 9999
FRAME_INTERVAL = 5
OCR_MAX_DISTANCE_M = 12.0

COLOR_CLUSTER = (0,255,255)    # 🟨 클러스터 박스: 노란색
COLOR_WORD    = (255,0,255)    # 🟣 단어 박스: 형광 핑크(마젠타)
COLOR_LINE    = (255,0,0)

# ===========================
#   전처리 함수
# ===========================
def light_normalize(bgr):
    ycrcb = cv2.cvtColor(bgr, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)
    y2 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4)).apply(y)
    return cv2.cvtColor(cv2.merge([y2, cr, cb]), cv2.COLOR_YCrCb2BGR)

def enhance_contrast(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    l = cv2.equalizeHist(l)
    return cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2BGR)

def all_preprocesses(img):
    return [("원본", img), ("밝기정규화", light_normalize(img)), ("대비강화", enhance_contrast(img))]

def similar(a,b):
    ratio = SequenceMatcher(None,a,b).ratio()
    if abs(len(a)-len(b)) > max(len(a),len(b))*0.5:
        return False
    return ratio > 0.9

# ===========================
#   방향 및 추천경로
# ===========================
def direction_from_bbox(x_center, w):
    if x_center < w/3: return "왼쪽"
    elif x_center < 2*w/3: return "앞쪽"
    else: return "오른쪽"

def recommend_path(obstacles):
    if not obstacles: return "직진"
    left  = any(o=="왼쪽" for o in obstacles)
    right = any(o=="오른쪽" for o in obstacles)
    front = any(o=="앞쪽" for o in obstacles)
    if front and not left: return "좌회전"
    elif front and not right: return "우회전"
    elif left and not right: return "우회전"
    elif right and not left: return "좌회전"
    else: return "직진"

# ===========================
#   GPT 교정 함수
# ===========================
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
def correct_text_with_api(sentence, distance, direction):
    prompt = f"""
    아래 문장은 OCR 인식 결과입니다. 오타나 비문을 교정하고,
    자연스러운 안내멘트 형식으로 완성해 주세요:
    "{direction} {distance:.1f}미터 거리에 '{{교정된 문장}}' 표지판이 감지되었습니다."
    OCR 문장: "{sentence}"
    """
    try:
        response = client.responses.create(model="gpt-5-mini", input=prompt)
        return response.output_text.strip()
    except Exception as e:
        print("⚠️ Lexicon 보정 실패:", e)
        return f"{direction} {distance:.1f}미터 거리에 \"{sentence}\" 표지판이 감지되었습니다."

# ===========================
#   모델 초기화
# ===========================
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO(YOLO_WEIGHTS, task="segment").to(device)
ocr = PaddleOCR(lang="korean", use_angle_cls=True, show_log=False)

# ===========================
#   RealSense 초기화
# ===========================
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(BAG_FILE, repeat_playback=False)
config.enable_stream(rs.stream.depth, rs.format.z16, 30)
config.enable_stream(rs.stream.color, rs.format.rgb8, 30)
profile = pipeline.start(config)
align = rs.align(rs.stream.color)
depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

try:
    pb = profile.get_device().as_playback()
    pb.set_real_time(False)
    print("✅ RealSense playback set to non-real-time mode.")
except: pass

# ===========================
#   메인 루프
# ===========================
frame_count = 0

try:
    plt.ioff()
    while True:
        try:
            frames = pipeline.wait_for_frames(timeout_ms=15000)
        except RuntimeError:
            print("⚠️ No more frames (end of bag or timeout).")
            break

        aligned = align.process(frames)
        dframe, cframe = aligned.get_depth_frame(), aligned.get_color_frame()
        if not dframe or not cframe:
            continue

        if frame_count < FRAME_START:
            frame_count += 1; continue
        if frame_count > FRAME_END: break
        if frame_count % FRAME_INTERVAL != 0:
            frame_count += 1; continue

        rgb = np.asanyarray(cframe.get_data())
        color = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        depth = np.asanyarray(dframe.get_data()) * depth_scale
        h0, w0 = color.shape[:2]

        # === (1) OCR 다중 전처리 후 통합 ===
        candidates=[]
        for _,proc in all_preprocesses(color):
            ocr_res = ocr.ocr(proc, det=True, rec=True, cls=False)
            if not ocr_res: continue
            for line in ocr_res[0]:
                box,(txt,conf)=line
                txt=re.sub(r'[^가-힣0-9a-zA-Z ]','',txt.strip())
                if not txt or conf<0.75: continue
                candidates.append((txt,conf,box))

        # === (2) 유사 단어 병합 ===
        merged=[]
        for txt,conf,box in candidates:
            found=False
            for m in merged:
                if similar(txt,m["txt"]):
                    if conf>m["conf"]: m.update({"txt":txt,"conf":conf,"box":box})
                    found=True; break
            if not found: merged.append({"txt":txt,"conf":conf,"box":box})

        if not merged:
            frame_count+=1; continue

        # === (3) OCR 단어 박스 표시 (형광 핑크) ===
        for txt, conf, box in candidates:
            pts = np.array(box).astype(int)
            cv2.polylines(color, [pts], isClosed=True, color=COLOR_WORD, thickness=2)

        # === (4) DBSCAN 문장 클러스터링 ===
        eps_candidates = [int(w0 * e) for e in [0.02,0.03,0.04,0.05]]
        best_eps, best_sil = None, -1
        centers = np.array([[np.mean(np.array(m["box"])[:,0]),
                             np.mean(np.array(m["box"])[:,1])] for m in merged])

        for eps_val in eps_candidates:
            clustering = DBSCAN(eps=eps_val, min_samples=1).fit(centers)
            if len(set(clustering.labels_)) > 1:
                try:
                    sil = silhouette_score(centers, clustering.labels_)
                    if sil > best_sil:
                        best_sil = sil
                        best_eps = eps_val
                except: continue

        eps_val = best_eps or int(w0 * 0.03)
        clustering = DBSCAN(eps=eps_val, min_samples=1).fit(centers)
        labels = clustering.labels_

        clusters=[]
        for l in set(labels):
            group=[merged[i] for i in range(len(merged)) if labels[i]==l]
            group.sort(key=lambda x: np.mean(np.array(x["box"])[:,0]))
            sentence=" ".join([g["txt"] for g in group])
            conf=np.mean([g["conf"] for g in group])
            all_pts=np.concatenate([np.array(g["box"]) for g in group],axis=0)
            x1,y1=np.min(all_pts,axis=0)
            x2,y2=np.max(all_pts,axis=0)
            clusters.append({"text":sentence,"conf":conf,"bbox":[x1,y1,x2,y2]})

        # === (5) GPT 교정 + 모든 클러스터 표시 ===
        frame_clusters = []
        warning_detected = False
        for cl in clusters:
            x1,y1,x2,y2=map(int,cl["bbox"])
            txt,conf=cl["text"],cl["conf"]
            cx=(x1+x2)/2
            direction=direction_from_bbox(cx,w0)
            roi=depth[y1:y2,x1:x2]
            vals=roi[(roi>0)&np.isfinite(roi)]
            if vals.size==0: continue
            dist=float(np.median(vals))
            if dist>OCR_MAX_DISTANCE_M: continue

            corrected = correct_text_with_api(txt,dist,direction)
            frame_clusters.append((txt, corrected, conf))
            cv2.rectangle(color,(x1,y1),(x2,y2),COLOR_CLUSTER,3)

            if any(w in txt or w in corrected for w in ["주의","위험","경고","낙하"]):
                warning_detected = True

        # === (6) YOLO + 추천경로 ===
        results=model.track(color,tracker="bytetrack.yaml",persist=True,conf=0.25)[0]
        boxes,ids=[],[]
        if getattr(results,"boxes",None) and results.boxes.id is not None:
            boxes=results.boxes.data.cpu().numpy()
            ids=results.boxes.id.cpu().numpy().astype(int)
        obstacle_dirs=[]
        for i in range(len(ids)):
            x_center=(boxes[i][0]+boxes[i][2])/2
            obstacle_dirs.append(direction_from_bbox(x_center,w0))

        path=recommend_path(obstacle_dirs) or "직진"
        if warning_detected:
            path = "주의"

        # === (7) 3분할 + 추천경로 표시 ===
        h,w,_=color.shape
        cv2.line(color,(w//3,0),(w//3,h),COLOR_LINE,2)
        cv2.line(color,(2*w//3,0),(2*w//3,h),COLOR_LINE,2)

        # === (8) 하단 텍스트 표시 ===
        img_pil = Image.fromarray(color)
        draw = ImageDraw.Draw(img_pil)
        font = ImageFont.truetype(FONT_PATH, 24)
        font_title = ImageFont.truetype(FONT_PATH, 32)
        font_label = ImageFont.truetype(FONT_PATH, 24)

        line_height = 55
        y_text = h0 - (line_height * len(frame_clusters)) - 20

        for i, (txt, corr, conf) in enumerate(frame_clusters):
            draw.text((25, y_text + i*line_height),     f"교정문: {corr}", font=font, fill=(255,255,0,255))
            draw.text((25, y_text + i*line_height + 25), f"원문: {txt}", font=font, fill=(100,200,255,255))

        # 🟡 추천 경로 = 클러스터 박스와 동일한 노란색
        draw.text((20, 20), f"추천 경로: {path}", font=font_title, fill=(0,255,255,255))

        # 🟧 좌/정면/우 = 형광 주황색
        ORANGE = (255,165,0,255)
        draw.text((w//6 - 40, 10), "좌", font=font_label, fill=ORANGE)
        draw.text((w//2 - 40, 10), "정면", font=font_label, fill=ORANGE)
        draw.text((5*w//6 - 40, 10), "우", font=font_label, fill=ORANGE)

        color = np.array(img_pil)

        # === 시각화 ===
        plt.figure(figsize=(10,6))
        plt.imshow(cv2.cvtColor(color,cv2.COLOR_BGR2RGB))
        plt.title(f"Frame {frame_count}")
        plt.axis("off"); plt.show()

        frame_count+=1
        gc.collect()

except KeyboardInterrupt:
    print("중단됨")
finally:
    try: pipeline.stop()
    except: pass
    gc.collect()
